# Summer Camp lab 3 - Time Series


New this lab:
* Trend
* Seasonality
* Lag
* Hybrid Models




**To work in the notebook, first copy the notebook to your own drive. File > "Save a copy in Drive"**

# Setting Up the Workspace

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns

#For timeseries
from sklearn.linear_model import LinearRegression

#Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import TimeSeriesSplit
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

plt.rc("figure", autolayout=True, figsize=(12, 4))

# Case Study

We will be using a dataset used in the Rossman Store Sales Kaggle competition. The competition was organized by Kaggle and sponsored by Rossmann, a drugstore chain in Germany. The purpose of the competition was to develop predictive models that could accurately forecast the sales of Rossmann stores based on historical data and various features.

The dataset includes information about individual Rossmann stores, such as the store's characteristics, the day of the week, whether there were promotions or holidays, and other relevant factors. Participants in the competition were tasked with building models that could predict the future sales of these stores, given the provided historical data.

The primary goals of the Rossman Store Sales competition were to encourage participants to apply machine learning techniques to real-world retail data and to develop models that could assist retailers like Rossmann in optimizing their operations, inventory management, and sales forecasting.


* Store: A unique identifier for each store in the dataset.

* DayOfWeek: The day of the week when the sale occurred (1 for Monday, 2 for Tuesday, and so on).

* Date: The date of the sale.

* Sales: The target variable, representing the sales for a particular store on a given day.

* Customers: The number of customers who made a purchase on a specific day.

* Open: A binary indicator (0 or 1) representing whether the store was open or closed on a given day.

* Promo: A binary indicator (0 or 1) representing whether a store was running a promotion on that day.

* StateHoliday: A categorical variable indicating whether it is a state holiday or not.

* SchoolHoliday: A binary indicator (0 or 1) representing whether it is a school holiday or not.

* StoreType: A categorical variable indicating the type of store (e.g., a, b, c, d).

* Assortment: A categorical variable describing the assortment level (e.g., a, b, c).

* CompetitionDistance: The distance to the nearest competitor store.



# Loading the Data

In [ ]:
#import pandas as pd
df_sales = pd.read_csv('https://www.dropbox.com/scl/fi/hxl03q2yp27kqmwgr8grr/rossman_sales.csv?rlkey=b8gmoasdgztukchq98zzzujk7&dl=1').drop(columns='DayOfWeek')
#Parse date
df_sales.Date = pd.to_datetime(df_sales.Date)
#Sort by Date and Store
df_sales = df_sales.sort_values(['Date','Store'])
#Set promo off if store is not open
df_sales.loc[df_sales["Open"] == 0, "Promo"] = 0

In [ ]:
df_sales

In [ ]:
df_sales.dtypes

In [ ]:
df_sales.head()

# Data Exploration

In [ ]:
df_sales.tail()

In [ ]:
df_sales.info()

In [ ]:
df_sales.describe().T

In [ ]:
#Plot daily data.
df_daily_sales = df_sales.groupby('Date')['Sales'].sum()
df_daily_sales.plot.line()

In [ ]:
#Sum daily data into weekly data and plot weekly sales.
df_weekly_sales = df_daily_sales.resample('W-Mon').sum()
df_weekly_sales.plot.line()

28 Day moving average.

In [ ]:
df_daily_sales.rolling(window=28, center=True).mean().plot.line()

We will focus on a single store for now:

In [ ]:
df_sales1 = df_sales[df_sales.Store == 1].set_index('Date')
df_sales1

28 day moving average

In [ ]:
df_sales1['Sales'].rolling(window=28, center=True).mean().plot.line()

# Trend

Pandas

In [ ]:
data = df_sales1.copy()
data['Day'] = np.arange(0, len(data.index))
data

In [ ]:
fig, ax = plt.subplots()
ax.plot('Day', 'Sales', data=data, color='0.75', alpha=0.4)
ax = sns.regplot(x='Day', y='Sales', data=data, ci=None, scatter_kws=dict(color='0.25'), marker="x")
ax.set_title('Time Plot of Daily Sales at Store 1');

StatsModels DeterministicProcess

In [ ]:
#Set period with the correct frequency (W for weekly) and DeterministicProcess will set the Date as index
data = df_sales1.copy().to_period('D')

dp = DeterministicProcess(
    index=data.index,  # dates from the training data
    constant=True,       # dummy feature for the bias (y_intercept)
    order=1,             # the time dummy (trend)
    drop=True,           # drop terms if necessary to avoid collinearity
)

y = data['Sales']
X = dp.in_sample()
X

Linear regression with timestep features.

In [ ]:
# The intercept is the same as the `const` feature from
# DeterministicProcess. LinearRegression behaves badly with duplicated
# features, so we need to be sure to exclude it here.
model = LinearRegression(fit_intercept=False)

# Train the model
model.fit(X, y)

# Store the fitted values as a time series with the same time index as
# the training data
y_pred = pd.Series(model.predict(X), index=X.index, name='Sales')
print (f"MAE: {mean_absolute_error(model.predict(X), y)}")

print (f"Coeffecient {model.coef_}")
print (f"Intercept {model.intercept_}")

In [ ]:
ax = y.plot(label="Sales")
ax = y_pred.plot(ax=ax, linewidth=3, label="Trend")
ax.set_title('Time Plot of Daily Sales at Store 1')
_ = ax.legend()

Predict out of sample

In [ ]:
# Timestamp features, 180 days out in the feature
X_fore = dp.out_of_sample(steps=180)
X_fore

In [ ]:
#Forecast
y_fore = pd.Series(model.predict(X_fore), index=X_fore.index).to_frame('Sales')

#Plot results
ax = data.Sales.plot(title="Time Plot of Daily Sales at Store 1 - Forecast")
ax = y_pred.plot(ax=ax, linewidth=3, label="Trend")
ax = y_fore.plot(ax=ax, linewidth=3, label="Trend Forecast", color="C3")
_ = ax.legend()

# Seasonality

## Code

In [ ]:
# annotations: https://stackoverflow.com/a/49238256/5769929
def seasonal_plot(X, y, period, freq, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    palette = sns.color_palette("husl", n_colors=X[period].nunique(),)
    ax = sns.lineplot(
        x=freq,
        y=y,
        hue=period,
        data=X,
        ci=False,
        ax=ax,
        palette=palette,
        legend=False,
    )
    ax.set_title(f"Seasonal Plot ({period}/{freq})")
    for line, name in zip(ax.lines, X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate(
            name,
            xy=(1, y_),
            xytext=(6, 0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=14,
            va="center",
        )
    return ax


## Example

In [ ]:
X = df_sales1.resample('W').sum().copy()

X["month"] = X.index.month  # the x-axis (freq)
X["year"] = X.index.year  # the seasonal period (period)

X["week"] = X.index.isocalendar().week
X["quarter"] = X.index.quarter

fig, ax = plt.subplots(1, 1, figsize=(11, 6))
seasonal_plot(X, y="Sales", period="year", freq="week", ax=ax)
plt.show()

In [ ]:
result = seasonal_decompose(data['Sales'].to_timestamp())
_ = result.plot()

In [ ]:
#Set period with the correct frequency (D for Daily, W for weekly) and DeterministicProcess will set the Date as index
data = df_sales1.copy().to_period('D')

dp = DeterministicProcess(
    index=data.index,   # dates from the training data
    constant=True,       # dummy feature for the bias (y_intercept)
    order=1,             # the time dummy (trend)
    drop=True,           # drop terms if necessary to avoid collinearity
    seasonal=True        # Daily seasonality (indicators)
)

y = data['Sales']
X = dp.in_sample()
X

Seasonal forecast with day-of-week indicators.

In [ ]:
y = data["Sales"]

model = LinearRegression(fit_intercept=False)
_ = model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=y.index)
print (f"MAE: {mean_absolute_error(y_pred, y)}")

X_fore = dp.out_of_sample(steps=90)
y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

ax = y.plot(color='0.25', style='.', title="Daily Sales at Store 1 - Seasonal Forecast")
ax = y_pred.plot(ax=ax, label="Seasonal")
ax = y_fore.plot(ax=ax, label="Seasonal Forecast", color='C3')
_ = ax.legend()

In [ ]:
#Set period with the correct frequency (D for Daily, W for weekly) and DeterministicProcess will set the Date as index
data = df_sales1.copy().to_period('D')
fourier = CalendarFourier(freq='Y', order=12)

dp = DeterministicProcess(
    index=data.index,  # dates from the training data
    constant=True,       # dummy feature for the bias (y_intercept)
    order=1,             # the time dummy (trend)
    drop=True,           # drop terms if necessary to avoid collinearity
    seasonal=True,       # Daily seasonality (indicators)
    additional_terms=[fourier],
)

y = data['Sales']
X = dp.in_sample()
X

In [ ]:
result = seasonal_decompose(data['Sales'].to_timestamp())
_ = result.plot()

In [ ]:
y = data["Sales"]

model = LinearRegression(fit_intercept=False)
_ = model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=y.index)
print (f"MAE: {mean_absolute_error(y_pred, y)}")

X_fore = dp.out_of_sample(steps=90)
y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

ax = y.plot(color='0.25', style='.', title="Daily Sales at Store 1 - Seasonal Forecast")
ax = y_pred.plot(ax=ax, label="Seasonal")
ax = y_fore.plot(ax=ax, label="Seasonal Forecast", color='C3')
_ = ax.legend()

## Weekly Sales Forecast

In [ ]:
#Set period with the correct frequency (D for Daily, W for weekly) and DeterministicProcess will set the Date as index
data = df_sales1.copy().resample('W-MON').sum(numeric_only=True).to_period('W')

dp = DeterministicProcess(
    index=data.index,   # dates from the training data
    constant=True,       # dummy feature for the bias (y_intercept)
    order=1,             # the time dummy (trend)
    drop=True,           # drop terms if necessary to avoid collinearity
    seasonal=True        # Daily seasonality (indicators)
)

X = dp.in_sample()
X

In [ ]:
result = seasonal_decompose(data['Sales'].to_timestamp())
plt.show()

In [ ]:
y = data["Sales"]

model = LinearRegression(fit_intercept=False)
_ = model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=y.index)
print (f"MAE: {mean_absolute_error(y_pred, y)}")
X_fore = dp.out_of_sample(steps=52)
y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

ax = y.plot(color='0.25', style='.', title="Weekly Sales at Store 1 - Seasonal Forecast")
ax = y_pred.plot(ax=ax, label="Seasonal")
ax = y_fore.plot(ax=ax, label="Seasonal Forecast", color='C3')
_ = ax.legend()

# Lag

Pandas provides us a simple method to lag a series, the `shift` method.

In [ ]:
data = df_sales1.copy().to_period('D')

#shift the daily sales column by 7 row. So that the lag_7 column of day 7 is the sales of one week prior.
data['Lag_7'] = data['Sales'].shift(7)
data[['Sales', 'Lag_7']].head(14)

When creating lag features, we need to decide what to do with the missing values produced. Filling them in is one option, maybe with 0.0 or "backfilling" with the first known value. Instead, we'll just drop the missing values, making sure to also drop values in the target from corresponding dates.

In [ ]:
#from sklearn.linear_model import LinearRegression

X = data[['Lag_7']].dropna() # drop missing values in the feature set
y = data['Sales']  # create the target
y, X = y.align(X, join='inner')  # drop corresponding values in target

model = LinearRegression()
model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=X.index)
print (f"MAE: {mean_absolute_error(y_pred, y)}")
print (f"Coeffecient {model.coef_}")
print (f"Intercept {model.intercept_}")

In [ ]:
fig, ax = plt.subplots()
ax.plot(X['Lag_7'], y, '.', color='0.25')
ax.plot(X['Lag_7'], y_pred)
ax.set_aspect('equal')
ax.set_ylabel('Sales')
ax.set_xlabel('Lag_7')
ax.set_title('Lag Plot of Store 1');

In [ ]:
_ = plot_acf(data.Sales)

In [ ]:
_ = plot_pacf(data.Sales)

In [ ]:
ax = y.plot()
ax = y_pred.plot(title='Daily sales at store 1, Linear model with lag features.')

In [ ]:
def lagplot(x, y=None, shift=1, standardize=False, ax=None, **kwargs):
    from matplotlib.offsetbox import AnchoredText
    x_ = x.shift(shift)
    if standardize:
        x_ = (x_ - x_.mean()) / x_.std()
    if y is not None:
        y_ = (y - y.mean()) / y.std() if standardize else y
    else:
        y_ = x
    corr = y_.corr(x_)
    if ax is None:
        fig, ax = plt.subplots()
    scatter_kws = dict(
        alpha=0.75,
        s=3,
    )
    line_kws = dict(color='C3', )
    ax = sns.regplot(x=x_,
                     y=y_,
                     scatter_kws=scatter_kws,
                     line_kws=line_kws,
                     lowess=True,
                     ax=ax,
                     **kwargs)
    at = AnchoredText(
        f"{corr:.2f}",
        prop=dict(size="large"),
        frameon=True,
        loc="upper left",
    )
    at.patch.set_boxstyle("square, pad=0.0")
    ax.add_artist(at)
    title = f"Lag {shift}" if shift > 0 else f"Lead {shift}"
    ax.set(title=f"Lag {shift}", xlabel=x_.name, ylabel=y_.name)
    return ax

#Function to plot lags
def plot_lags(x,
              y=None,
              lags=6,
              leads=None,
              nrows=1,
              lagplot_kwargs={},
              **kwargs):
    import math
    kwargs.setdefault('nrows', nrows)
    orig = leads is not None
    leads = leads or 0
    kwargs.setdefault('ncols', math.ceil((lags + orig + leads) / nrows))
    kwargs.setdefault('figsize', (kwargs['ncols'] * 2, nrows * 2 + 0.5))
    fig, axs = plt.subplots(sharex=True, sharey=True, squeeze=False, **kwargs)
    for ax, k in zip(fig.get_axes(), range(kwargs['nrows'] * kwargs['ncols'])):
        k -= leads + orig
        if k + 1 <= lags:
            ax = lagplot(x, y, shift=k + 1, ax=ax, **lagplot_kwargs)
            title = f"Lag {k + 1}" if k + 1 >= 0 else f"Lead {-k - 1}"
            ax.set_title(title, fontdict=dict(fontsize=14))
            ax.set(xlabel="", ylabel="")
        else:
            ax.axis('off')
    plt.setp(axs[-1, :], xlabel=x.name)
    plt.setp(axs[:, 0], ylabel=y.name if y is not None else x.name)
    fig.tight_layout(w_pad=0.1, h_pad=0.1)
    return fig


In [ ]:
_ = plot_lags(y,lags=7)

# Deseasoning

In [ ]:
#Set period with the correct frequency (D for Daily, W for weekly) and DeterministicProcess will set the Date as index
data = df_sales1.copy().to_period('D')

fourier = CalendarFourier(freq='Y', order=12)

dp = DeterministicProcess(
    index=data.index,  # dates from the training data
    constant=True,       # dummy feature for the bias (y_intercept)
    order=1,             # the time dummy (trend)
    drop=True,           # drop terms if necessary to avoid collinearity
    seasonal=True,       # Daily seasonality (indicators)
    additional_terms=[fourier],
)

y = data['Sales']
X_time = dp.in_sample()

model = LinearRegression(fit_intercept=False)
model.fit(X_time, y)
print (f"MAE: {mean_absolute_error(model.predict(X_time), y)}")

y_deseason = y - model.predict(X_time)
y_deseason.name = 'sales_deseasoned'

ax = y_deseason.plot()
ax.set_title("Sales at store 1 (deseasonalized / residuals)");

In [ ]:
ax = y_deseason.plot()
plt.plot_date(data[data.Open == 0].index, y_deseason[data[data.Open == 0].index], color='C3')
ax.set_title('Residual Sales - Store not open');
plt.rc("figure", autolayout=True, figsize=(11, 4))

We see that a large number of negative outliers in our residuals occur when Open=0, likely indicating holidays that we have not included in our seaonal model so far.

In [ ]:
y_deseason.to_frame('residuals').merge(data,left_index=True,right_index=True).plot.box(column='residuals', by='Open', grid=False,title='Residuals by Open value')

In [ ]:
ax = y_deseason.plot()
plt.plot_date(data[(data.Promo == 1)].index, y_deseason[data[(data.Promo == 1)].index], color='C2')
ax.set_title('Store has promotion');
plt.rc("figure", autolayout=True, figsize=(11, 4))

Typically we see that we see different residuals depending on whether promotion is active.

In [ ]:
y_deseason.to_frame('residuals').merge(data,left_index=True,right_index=True).plot.box(column='residuals', by='Promo', grid=False,title='Residuals by Promotion value')

In [ ]:
_ = plot_lags(y_deseason,lags=7)

# Exogenous Factors


In [ ]:
X2 = X_time.join(data[['Open','Promo']])
X2

In [ ]:
model = LinearRegression().fit(X2, y)
y_pred = pd.Series(
    model.predict(X2),
    index=X2.index,
    name='Fitted',
)

y_pred = pd.Series(model.predict(X2), index=X2.index).clip(0.0)
print (f"MAE: {mean_absolute_error(y_pred, y)}")
ax = y.plot(alpha=0.5, title="Sales at store 1, with promo and open factor", ylabel="items sold")
ax = y_pred.plot(ax=ax, label="Seasonal")
ax.legend();

# Cross Validation



In [ ]:
ts_cv_scores = cross_val_score(model, X2, y_pred, cv=TimeSeriesSplit(n_splits=4), scoring='neg_mean_absolute_error') * -1
print (f'time series cross validation Mean score {ts_cv_scores.mean()}')
ts_cv_scores

## Train test split

So far we only looked at statistical models, where we don't necessarily need a test/validation dataset.

You can use train_test_split to split up your dataset into train and test sets. However you **NEED to set `shuffle=False`** to avoid data being shuffled and losing your temporal relationships in your data.

In [ ]:
y = data['Sales']
X = data[['Promo','Open']]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, shuffle=False)
X_train

You might want to consider applying a manual split to break off at a more natural breakpoint, for example making sure that both train and test set start at a Monday


In [ ]:
from_date = '2013-01-01' #Tuesday
to_date = '2015-01-06' #Tuesday
X_train = X[from_date:to_date]
X_test = X[to_date:]
y_train = y[from_date:to_date]
y_test = y[to_date:]
print (f"Length train {len(X_train)} test {len(X_test)}")

# Hybrid models



###### Code

In [ ]:
class BoostedHybrid:
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2 # Model 2 trains on the residuals of the forecast of model_1
        self.y_columns = None  # store column names from fit method


    def fit(self, X_1, X_2, y):
        # Train model_1
        self.model_1.fit(X_1, y)

        # Make predictions
        y_fit = pd.DataFrame(
            self.model_1.predict(X_1),
            index=X_1.index, columns=y.columns,
        )

        # Compute residuals
        y_resid = y - y_fit
        y_resid = y_resid.stack().squeeze() # wide to long

        # Train model_2 on residuals
        self.model_2.fit(X_2, y_resid)

        # Save column names for predict method
        self.y_columns = y.columns

        self.y_fit = y_fit
        self.y_resid = y_resid

    def predict(self, X_1, X_2):
        # Predict with model_1
        y_pred = pd.DataFrame(
            self.model_1.predict(X_1),
            index=X_1.index, columns=self.y_columns,
        )
        y_pred = y_pred.stack().squeeze()  # wide to long

        # Add model_2 predictions to model_1 predictions
        y_pred += self.model_2.predict(X_2)

        return y_pred.unstack()

#### Example

In [ ]:
# Create model
model = BoostedHybrid(
    model_1=LinearRegression(),
    model_2=RandomForestRegressor(),
)

X_1 = X_time
X_2 = data[['Open','Promo']]
model.fit(X_1, X_2, data[['Sales']])

y_pred = model.predict(X_1, X_2)
y_pred = y_pred.clip(0.0) #.clip(0.0) removes negative values
print (f"MAE: {mean_absolute_error(y_pred.Sales, y)}")

In [ ]:
ax = y.plot(alpha=0.5, title="Sales at store 1, with hybrid model", ylabel="items sold")
ax = y_pred.Sales.plot(ax=ax, label="Seasonal")
ax.legend();

# Assignment

1) Repeat the lab notebook experimenting with the following Time-Step features and Time Series features

* Time-Step feature: Fourier Series. Instead of 12 monthly sine-cosine pairs over the year, replace with 4 sine-cosine pairs

* Time Series feature: Add Lag and Lead for promotion.


Does the addition of the new Time-Step and Time Series features yield "better" results than in lab? Explain your intuition why or why not.

2) Experiment with a different linear model (lasso, ridge, elasticnet) and a different non-linear model (gradient boosted trees) in a new hybrid model.


Do your changes of the linear model (model 1) and the non-linear model (model 2) yield "better" results than in lab? Explain your intuition why or why not.
